In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from pandas.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.optimize import minimize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
## !pip install 'polars[numpy,pandas,pyarrow]' --target=/kaggle/working/mysitepackages

!pip download polars[numpy,pandas,pyarrow] -d /kaggle/working/mysitepackages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 44.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 56.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 31.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 82.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 35.3 MB/s eta 0:00:00
Saved ./mysitepackages/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Saved ./mysitepackages/pyarrow-11.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/typing_extensions-4.5.0-py3-none-any.whl
Saved ./mysitepackages/pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/polars-0.17.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/python_da

#### create the zip file with the code below and then download the zip file into local machine

In [3]:
from zipfile import ZipFile

dirName = "/kaggle/working/mysitepackages"
zipName = "packages.zip"

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)

In [5]:
! ls /kaggle/input/packages-to-install-polars-offline/kaggle/working/mysitepackages

numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
polars-0.17.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pyarrow-11.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
python_dateutil-2.8.2-py2.py3-none-any.whl
pytz-2023.3-py2.py3-none-any.whl
six-1.16.0-py2.py3-none-any.whl
typing_extensions-4.5.0-py3-none-any.whl


#### Upload the downloaded zipfile from local machine and then fire the command below

In [2]:
!pip install polars[numpy,pandas,pyarrow] --no-index --find-links=file:///kaggle/input/packages-to-install-polars-offline/kaggle/working/mysitepackages/ 

Looking in links: file:///kaggle/input/packages-to-install-polars-offline/kaggle/working/mysitepackages/


In [2]:
import polars as pl

In [9]:
print(defog_train_data.select('Id').n_unique())
print(defog_train_data.estimated_size("mb"))
print(defog_train_data.shape)

91
291.12402725219727
(4111322, 10)


In [10]:
defog_train_data.head(10)

Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id
i64,f64,f64,f64,i64,i64,i64,bool,bool,str
1000,-1.015138,-0.071969,0.161124,0,0,0,true,true,"""be9d33541d"""
1001,-1.018566,-0.05785,0.144988,0,0,0,true,true,"""be9d33541d"""
1002,-1.037102,-0.0542,0.129327,0,0,0,true,true,"""be9d33541d"""
1003,-1.036416,-0.060578,0.115451,0,0,0,true,true,"""be9d33541d"""
1004,-1.012407,-0.069198,0.110121,0,0,0,true,true,"""be9d33541d"""
1005,-0.98234,-0.078566,0.113947,0,0,0,true,true,"""be9d33541d"""
1006,-0.962148,-0.088102,0.121993,0,0,0,true,true,"""be9d33541d"""
1007,-0.947028,-0.100937,0.129277,0,0,0,true,true,"""be9d33541d"""
1008,-0.938086,-0.105582,0.136236,0,0,0,true,true,"""be9d33541d"""


In [37]:
defog_metadata = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv", try_parse_dates=True)\
.collect()
print(defog_metadata.shape)
print(defog_metadata.select('Subject').n_unique())
print(defog_metadata.select('Id').n_unique())
defog_metadata.filter(pl.col('Subject')=='bf608b')

(137, 4)
45
137


Id,Subject,Visit,Medication
str,str,i64,str
"""02ea782681""","""bf608b""",2,"""on"""
"""4c0aec9035""","""bf608b""",2,"""off"""
"""8228c6fdee""","""bf608b""",1,"""on"""
"""a8c5a489bd""","""bf608b""",1,"""off"""


In [38]:
subjects = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv", try_parse_dates=True)\
.collect()
print(subjects.shape)
print(subjects.select('Subject').n_unique())
subjects.filter(pl.col('Subject')=='bf608b')

(173, 8)
136


Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
str,i64,i64,str,f64,i64,i64,i64
"""bf608b""",1,67,"""M""",7.0,15,48,23
"""bf608b""",2,67,"""M""",7.0,14,57,19


In [39]:
defog_subjects_metadata = defog_metadata.join(subjects, on=["Subject","Visit"], how="inner")

defog_subjects_metadata = defog_subjects_metadata.with_columns(pl.when(pl.col('Medication') == 'on').then(pl.col('UPDRSIII_On')).otherwise(pl.col('UPDRSIII_Off')).alias("UPDRSIII"))
defog_subjects_metadata = defog_subjects_metadata.drop(['UPDRSIII_On','UPDRSIII_Off'])

del defog_metadata, subjects

print(defog_subjects_metadata.shape)
print(defog_subjects_metadata.select('Subject').n_unique())
print(defog_subjects_metadata.select('Id').n_unique())
defog_subjects_metadata.filter(pl.col('Subject')=='bf608b')

(137, 9)
45
137


Id,Subject,Visit,Medication,Age,Sex,YearsSinceDx,NFOGQ,UPDRSIII
str,str,i64,str,i64,str,f64,i64,i64
"""8228c6fdee""","""bf608b""",1,"""on""",67,"""M""",7.0,23,15
"""a8c5a489bd""","""bf608b""",1,"""off""",67,"""M""",7.0,23,48
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,19,14
"""4c0aec9035""","""bf608b""",2,"""off""",67,"""M""",7.0,19,57


In [41]:
events = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv", try_parse_dates=True)\
.collect()
print(events.shape)
print(events.select('Id').n_unique())
events.filter(pl.col('Id').is_in(['a8c5a489bd']))

(3712, 5)
535


Id,Init,Completion,Type,Kinetic
str,f64,f64,str,i64
"""a8c5a489bd""",377.635,387.785,"""Turn""",1
"""a8c5a489bd""",388.35,400.676,"""Turn""",1
"""a8c5a489bd""",401.31,412.247,"""Turn""",1
"""a8c5a489bd""",413.043,427.845,"""Turn""",1
"""a8c5a489bd""",500.393,516.432,"""Turn""",1
"""a8c5a489bd""",527.695,552.497,"""Turn""",1
"""a8c5a489bd""",721.332,721.385,"""Turn""",1
"""a8c5a489bd""",833.732,834.267,"""Walking""",1


In [52]:
temp_defog = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog/a8c5a489bd.csv").collect()
temp_defog.filter((pl.col('Time')>=np.round(377.635 * 100)) & (pl.col('Time')<=np.round(387.785 * 100)))

Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task
i64,f64,f64,f64,i64,i64,i64,bool,bool
37764,-0.995575,-0.179791,-0.394888,0,1,0,true,true
37765,-1.042193,-0.161341,-0.40625,0,1,0,true,true
37766,-1.079852,-0.161897,-0.398695,0,1,0,true,true
37767,-1.103,-0.178719,-0.390625,0,1,0,true,true
37768,-1.106073,-0.19499,-0.377127,0,1,0,true,true
37769,-1.084005,-0.197246,-0.336168,0,1,0,true,true
37770,-1.057739,-0.161148,-0.328125,0,1,0,true,true
37771,-1.038785,-0.140625,-0.329622,0,1,0,true,true
37772,-0.993575,-0.141419,-0.347003,0,1,0,true,true


In [53]:
tasks = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv", try_parse_dates=True)\
.collect()
print(tasks.shape)
print(tasks.select('Id').n_unique())
tasks.filter(pl.col('Id').is_in(['a8c5a489bd']))

(2817, 4)
137


Id,Begin,End,Task
str,f64,f64,str
"""a8c5a489bd""",10.0,27.007,"""4MW"""
"""a8c5a489bd""",108.736,122.936,"""TUG-ST"""
"""a8c5a489bd""",139.84,153.47,"""TUG-ST"""
"""a8c5a489bd""",197.902,211.338,"""Turning-DT"""
"""a8c5a489bd""",239.781,252.631,"""TUG-DT"""
"""a8c5a489bd""",375.673,428.687,"""Turning-ST"""
"""a8c5a489bd""",488.872,553.165,"""Turning-DT"""
"""a8c5a489bd""",708.685,727.707,"""Hotspot1"""
"""a8c5a489bd""",824.692,839.145,"""Hotspot2"""


In [9]:
defog_subjects_events_tasks_metadata = defog_subjects_events_metadata.join(tasks, on=["Id"], how="inner", suffix="_right")

del defog_subjects_events_metadata, tasks

print(defog_subjects_events_tasks_metadata.shape)
print(defog_subjects_events_tasks_metadata.select('Id').n_unique())
print(defog_subjects_events_tasks_metadata.select('Subject').n_unique())
defog_subjects_events_tasks_metadata.head(10)

(46346, 17)
124
44


Id,Subject,Visit,Medication,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Begin,End,Task
str,str,i64,str,i64,str,f64,i64,i64,i64,f64,f64,str,i64,f64,f64,str
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,1969.173,1972.535,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2078.982,2080.524,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2290.606,2291.952,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2704.292,2704.77,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2785.534,2785.774,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2785.786,2786.344,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2827.264,2829.428,null,null,10.0,190.48,"""Rest1"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,1969.173,1972.535,null,null,211.24,271.56,"""Rest2"""
"""02ab235146""","""ab54e1""",2,"""on""",79,"""F""",8.0,38,64,28,2078.982,2080.524,null,null,211.24,271.56,"""Rest2"""


In [10]:
defog_subjects_events_tasks_metadata.filter(pl.col('Id')=='02ea782681')

Id,Subject,Visit,Medication,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Begin,End,Task
str,str,i64,str,i64,str,f64,i64,i64,i64,f64,f64,str,i64,f64,f64,str
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1394.649,1395.563,"""Turn""",1,10.0,21.618,"""4MW"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1483.884,1484.262,"""Turn""",1,10.0,21.618,"""4MW"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1394.649,1395.563,"""Turn""",1,99.404,111.2,"""4MW-C"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1483.884,1484.262,"""Turn""",1,99.404,111.2,"""4MW-C"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1394.649,1395.563,"""Turn""",1,174.44,177.909,"""MB1"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1483.884,1484.262,"""Turn""",1,174.44,177.909,"""MB1"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1394.649,1395.563,"""Turn""",1,202.02,209.735,"""MB2b"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1483.884,1484.262,"""Turn""",1,202.02,209.735,"""MB2b"""
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,14,57,19,1394.649,1395.563,"""Turn""",1,243.977,266.828,"""MB3-L"""


In [ ]:
defog_train_data = pl.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog'):
    for filename in filenames:
        
        temp = pl.scan_csv(os.path.join(dirname, filename), try_parse_dates=True)\
        .filter((pl.col("Valid")) & (pl.col("Task")))\
        .with_columns(pl.lit(filename[0:-4]).alias('Id'))\
        .collect()
        
        temp_final = temp.join(defog_subjects_events_tasks_metadata, on=["Id"], how="inner")
        del temp
        
        defog_train_data = pl.concat([defog_train_data, temp_final], how="vertical")

In [ ]:
defog_final = defog_train_data.join(defog_subjects_events_tasks_metadata, on=["Id"], how="inner", suffix="_right")

del defog_subjects_events_tasks_metadata, defog_train_data

print(defog_final.shape)
print(defog_final.select('Id').n_unique())
print(defog_final.select('Subject').n_unique())
defog_final.head(10)